<a href="https://colab.research.google.com/github/ufrpe-ensino/curso-mineracao-textos/blob/master/08_TopicModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem de Tópicos com LDA

## O que é modelagem de tópicos?
A modelagem de tópicos pode ser descrita como um método para localizar um grupo de palavras (ou seja, um tópico) de uma coleção de documentos (um "corpus") que melhor representa as informações na coleção. Extrair automaticamente o contexto de grandes corpos de dados e dividi-los em tópicos para uma análise mais detalhada é uma das principais aplicações do Processamento de Linguagem Natural.

## O que é um tópico?
Um tópico é um padrão repetido de termos coocorrentes em um corpus.

## O que é *Latent Dirichlet Allocation* (LDA)?

O LDA assume que os documentos são produzidos a partir de uma mistura de tópicos. Esses tópicos geram palavras com base em uma distribuição de probabilidade (LDA é um modelo estatístico generativo).

Dado um conjunto de dados de documentos, o LDA tenta descobrir quais tópicos criariam esses documentos em primeiro lugar.

# Importando dados

Base de dados de análise de sentimentos em português, contendo 8199 Tweets classificadas como positivo, negativo e neutro. 

https://minerandodados.com.br/analise-de-sentimentos-twitter-como-fazer/



In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/minerandodados/mdrepo/master/Tweets_Mg.csv'
df = pd.read_csv(url, encoding='utf-8')
df.head()

# Scikit-learn

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV

vectorizer = CountVectorizer(analyzer='word',
                             min_df=10,                        # Mínimo de ocorrências da palavra
                             lowercase=True,                   # converte para lowercase
                             token_pattern='[a-zA-Z0-9]{4,}',  # palavras com pelo menos 3 caracteres
                             # max_features=50000,             # número máximo de palavras
                            )

tokens_cv = vectorizer.fit_transform(df['Text'])
tokens_cv

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,           # Número de tópicos
                                      max_iter=10,               # Número de interações
                                      learning_method='online',
                                      random_state=100,          
                                      batch_size=128,            
                                      evaluate_every = -1,       
                                      n_jobs = -1,               # Número de CPUs
                                     )
lda_output = lda_model.fit_transform(tokens_cv)

In [ ]:
import numpy as np

def mostrar_topicos(vectorizer=vectorizer, lda_model='', n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = mostrar_topicos(vectorizer=vectorizer, lda_model=lda_model, n_words=15)

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Palavra '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Tópico '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

# Preprocessamento

Tokenização utilizando o [TweetTokenizer](https://www.nltk.org/api/nltk.tokenize.html) do NLTK. 

Tratamento diferenciado de smileys:


In [ ]:
df.iloc[147]['Text']

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
nltk.word_tokenize(df.iloc[147]['Text'])

In [ ]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer(strip_handles=False,  # remover 'mentions'
                                 preserve_case=False)
tweet_tokenizer.tokenize(df.iloc[147]['Text'])

In [ ]:
import re
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer(strip_handles=True, 
                    reduce_len=True, 
                    preserve_case=False)
def preprocessamento(text, join=True):
    #remove links, pontos, virgulas,ponto e virgulas dos tweets
    #coloca tudo em minusculo
    text = re.sub(r"http\S+", "", text).lower().replace(',','').replace('.','').replace(';','').replace('-','').replace(':','')
    if join:
      text = ' '.join(tt.tokenize(text))
    else:
      text = tt.tokenize(text)
    return text

tokens_nltk = vectorizer.fit_transform(df['Text'].apply(preprocessamento))
tokens_nltk

In [ ]:
lda_output = lda_model.fit_transform(tokens_nltk)

topic_keywords = mostrar_topicos(vectorizer=vectorizer, lda_model=lda_model, n_words=15)

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Palavra '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Tópico '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

# Visualizando com o pyLDAvis

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.sklearn
import pyLDAvis
import matplotlib.pyplot as plt
#%matplotlib inline

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, 
                                 tokens_nltk, 
                                 vectorizer, 
                                 mds='tsne', 
                                 sort_topics=False)
panel

## Salvando como HTML

In [ ]:
pyLDAvis.save_html(panel, 'LDA.html')

# Gensim

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import Phrases

docs = df['Text'].apply(lambda x: preprocessamento(x, join=False)).values

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

# Create a dictionary representation of the documents.
# docs = [' '.join(tokens) for tokens in tokens_nltk]
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

In [ ]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
model.show_topics()

In [ ]:
import pyLDAvis.gensim

panel = pyLDAvis.gensim.prepare(model, corpus, dictionary)
panel